In [25]:
from lyricsgenius import Genius
import pandas as pd
import os
from numpy import nan
import re
from tqdm import tqdm
from multiprocessing import Process
from multiprocessing.managers import SharedMemoryManager
from multiprocessing import Queue

In [13]:
df_ranking = pd.read_csv("data/ranking.csv")
df_ranking.head(5)

,year,rank,title,artist,modified_title,modified_artist
0,1946,1,"""Prisoner of Love""",Perry Como,Prisoner of Love,Perry Como
1,1946,2,"""To Each His Own""",Eddy Howard,To Each His Own,Eddy Howard
2,1946,3,"""The Gypsy""",The Ink Spots,The Gypsy,The Ink Spots
3,1946,4,"""Five Minutes More""",Frank Sinatra,Five Minutes More,Frank Sinatra
4,1946,5,"""Rumors Are Flying""",Frankie Carle,Rumors Are Flying,Frankie Carle


In [ ]:
# 全体がアーティスト名
# Peter, Paul and Mary
# Dino, Desi & Billy
# Blood, Sweat & Tears
# Checkmates, Ltd.
# Crosby, Stills, Nash & Young
# Hamilton, Joe Frank & Reynolds
# Earth, Wind & Fire
# Crosby, Stills & Nash
# Ray, Goodman & Brown
# 10,000 Maniacs


In [ ]:
# Earth, Wind & Fire & The Emotions
# Grover Washington, Jr. & Bill Withers
# Lil Wayne, Wiz Khalifa and Imagine Dragons with Logic and Ty Dolla Sign featuring X Ambassadors
# Nio García, Darell and Casper Mágico featuring...

In [ ]:
# 1986, 1, "Dionne and Friends (Dionne Warwick, Gladys Knight, Elton John and Stevie Wonder)"

In [20]:
# Bryan Adams, Rod Stewart and Sting
# Nelly, P. Diddy and Murphy Lee
# Nelly, Paul Wall and Ali & Gipp
# Eminem, Dr. Dre and 50 Cent
# Jessie J, Ariana Grande and Nicki Minaj
# Rihanna, Kanye West and Paul McCartney
# Jessie J, Ariana Grande and Nicki Minaj
# Cardi B, Bad Bunny and J Balvin
# Zedd, Maren Morris and Grey
# Migos, Nicki Minaj and Cardi B
# Benny Blanco, Halsey and Khalid
# Benny Blanco, Halsey and Khalid
# Gucci Mane, Bruno Mars and Kodak Black
# Young Thug, J. Cole and Travis Scott
# Cardi B, Bad Bunny and J Balvin
# Kane Brown, Swae Lee and Khalid
# Neiked, Mae Muller and Polo G
# Metro Boomin, the Weeknd and 21 Savage
# Metro Boomin, Future and Chris Brown

In [ ]:
# Christina Aguilera, Lil' Kim, Mýa and Pink
# Kanye West, Big Sean, Pusha T and 2 Chainz
# Jay Rock, Kendrick Lamar, Future and James Blake

In [ ]:
# Carolina Gaitán, Mauro Castillo, Adassa, Rhenzy Feliz, Diane Guerrero, Stephanie Beatriz and the Encanto cast

In [ ]:
# Lil' Kim featuring Da Brat, Left Eye, Missy El...
# Master P featuring Fiend, Silkk the Shocker, M...
# Blackstreet featuring Mýa, Mase and Blinky Blink
# Missy Elliott featuring Nas, Eve and Q-Tip
# P. Diddy featuring Ginuwine, Loon and Mario Wi...
# Ja Rule featuring Ashanti, Charli Baltimore an...
# Nelly featuring Murphy Lee, Ali, and Kyjuan
# Houston featuring Chingy, Nate Dogg and I-20


In [22]:
artists_exception1 = ["Peter, Paul and Mary", "Dino, Desi & Billy", "Blood, Sweat & Tears", "Checkmates, Ltd.", 
                     "Crosby, Stills, Nash & Young", "Hamilton, Joe Frank & Reynolds", 
                     "Earth, Wind & Fire", "Crosby, Stills & Nash", "Ray, Goodman & Brown", "10,000 Maniacs"]
artists_exception2 = ["Earth, Wind & Fire & The Emotions", 
                      "Grover Washington, Jr. & Bill Withers", 
                      "Dionne and Friends (Dionne Warwick, Gladys Knight, Elton John and Stevie Wonder)"]

In [24]:
r, c = df_ranking.shape
for i in range(r):
    artist = df_ranking.iloc[i, 3]
    if artist.find("(") != -1:
        print(df_ranking.iloc[i,])

year                                     1970
rank                                       84
title              "Somebody's Been Sleeping"
artist               100 Proof (Aged in Soul)
modified_title       Somebody's Been Sleeping
modified_artist      100 Proof (Aged in Soul)
Name: 1672, dtype: object
year                                                            1986
rank                                                               1
title                                  "That's What Friends Are For"
artist             Dionne and Friends (Dionne Warwick, Gladys Kni...
modified_title                           That's What Friends Are For
modified_artist                                               Dionne
Name: 3189, dtype: object
year                                                            2015
rank                                                              19
title                                              "Where Are Ü Now"
artist             Skrillex and Diplo (Jack Ü) feat

In [23]:
def list_find(l, x):
    if x in l:
        return True
    else:
        return False

r, c = df_ranking.shape
for i in range(r):
    artist = df_ranking.iloc[i, 3]
    if list_find(artists_exception1 + artists_exception2, artist):
        print(df_ranking.iloc[i,])

year                               1962
rank                                 98
title               "If I Had a Hammer"
artist             Peter, Paul and Mary
modified_title        If I Had a Hammer
modified_artist             Peter, Paul
Name: 885, dtype: object
year                                   1963
rank                                     12
title              "Puff, the Magic Dragon"
artist                 Peter, Paul and Mary
modified_title       Puff, the Magic Dragon
modified_artist                 Peter, Paul
Name: 899, dtype: object
year                                1963
rank                                  13
title              "Blowin' in the Wind"
artist              Peter, Paul and Mary
modified_title       Blowin' in the Wind
modified_artist              Peter, Paul
Name: 900, dtype: object
year                             1965
rank                               97
title                    "I'm a Fool"
artist             Dino, Desi & Billy
modified_title         

In [26]:
genius = Genius(access_token=os.environ["GeniusAPIClientAccessToken"], 
                verbose=False, 
                remove_section_headers=True, 
                retries=2)

In [3]:
def lyric_modification(lyric, pattern):
    modified_lyric = pattern.sub("|", lyric)
    return modified_lyric

In [7]:
r, c = df_ranking.shape
lyrics = [""] * r
search_error = [[] for _ in range(6)]
pattern = re.compile("\n+")

for i in tqdm(range(r)):
    year = df_ranking.iloc[i, 0]
    rank = df_ranking.iloc[i, 1]
    title = df_ranking.iloc[i, 2]
    artist = df_ranking.iloc[i, 3]
    modified_title = df_ranking.iloc[i, 4]
    modified_artist = df_ranking.iloc[i, 5]
    song_info = [year, rank, title, artist, modified_title, modified_artist]
    search_error_flag = False
    try:
        # インスト曲で歌詞を含まない場合はNoneが返される
        song = genius.search_song(title=modified_title, artist=modified_artist, get_full_info=False)
    except:
        # 検索に失敗した場合はNoneとする
        song = None
        search_error_flag = True
        for j in range(6):
            search_error[j].append(song_info[j])
    if song is not None:
        lyrics[i] = lyric_modification(song.lyrics, pattern)
    else:
        if search_error_flag:
            lyrics[i] = nan
        else:
            lyrics[i] = "no_lyric"

  0%|          | 0/6989 [00:00<?, ?it/s]

  0%|          | 19/6989 [00:35<5:01:13,  2.59s/it]

In [31]:
song = genius.search_song(title="Somebody's Been Sleeping", artist="", get_full_info=False)

In [32]:
song.artist

'100 Proof (Aged in Soul)'

In [18]:
pattern = re.compile("\n+")

In [19]:
lyric_modification(song.lyrics, pattern)

'(Yeah, yeah, yeah, yeah)|Fever dream high in the quiet of the night|You know that I caught it (Oh yeah, you\'re right, I want it)|Bad, bad boy, shiny toy with a price|You know that I bought it (Oh yeah, you\'re right, I want it)|Killing me slow, out the window|I\'m always waiting for you to be waiting below|Devils roll the dice, angels roll their eyes|What doesn\'t kill me makes me want you more|And it\'s new, the shape of your body|It\'s blue, the feeling I\'ve got|And it\'s ooh, woah-oh|It\'s a cruel summer|It\'s cool, that\'s what I tell \'em|No rules in breakable heaven|But ooh, woah-oh|It\'s a cruel summer with you|Hang your head low in the glow of the vending machine|I\'m not dying (Oh yeah, you\'re right, I want it)|We say that we\'ll just screw it up in these trying times|We\'re not trying (Oh yeah, you\'re right, I want it)|So cut the headlights, summer\'s a knife|I\'m always waiting for you just to cut to the bone|Devils roll the dice, angels roll their eyes|And if I bleed, 

In [12]:
print(a)

(Yeah, yeah, yeah, yeah)

Fever dream high in the quiet of the night
You know that I caught it (Oh yeah, you're right, I want it)
Bad, bad boy, shiny toy with a price
You know that I bought it (Oh yeah, you're right, I want it)

Killing me slow, out the window
I'm always waiting for you to be waiting below
Devils roll the dice, angels roll their eyes
What doesn't kill me makes me want you more

And it's new, the shape of your body
It's blue, the feeling I've got
And it's ooh, woah-oh
It's a cruel summer
It's cool, that's what I tell 'em
No rules in breakable heaven
But ooh, woah-oh
It's a cruel summer with you

Hang your head low in the glow of the vending machine
I'm not dying (Oh yeah, you're right, I want it)
We say that we'll just screw it up in these trying times
We're not trying (Oh yeah, you're right, I want it)

So cut the headlights, summer's a knife
I'm always waiting for you just to cut to the bone
Devils roll the dice, angels roll their eyes
And if I bleed, you'll be the la

In [1]:
from multiprocessing import Pool

In [1]:
import time
from multiprocessing import Pool

# 並列処理させる関数
def nijou(x):
    print('input: %d' % x)
    time.sleep(2)
    retValue = x * x
    print('double: %d' % (retValue))
    return(retValue)


p = Pool(4) # プロセス数を4に設定
result = p.map(nijou, range(10))  # nijou()に0,1,..,9を与えて並列演算
print(result)


In [33]:
from difflib import SequenceMatcher

SequenceMatcher(None, "taylor swift", "TAYLOR SWIFT").ratio()

0.08333333333333333

In [34]:
char1 = "A and B & C with D, E"
char1 = char1.replace(" and ", "|").replace(" & ", "|").replace(" with ", "|").replace(", ", "|")
artists = char1.split(sep="|")
artists

['A', 'B', 'C', 'D', 'E']

In [11]:
data = [[1, 2], [3, 4]]
df = pd.DataFrame(data=data)
df

,0,1
0,1,2
1,3,4


In [12]:
df = pd.read_csv("wrong_result0~9.csv")
df

,year,rank,title,artist,modified_title,modified_artist,lyric
0,1946,6,"""Oh! What It Seemed to Be""",Frankie Carle,Oh! What It Seemed to Be,Frankie Carle,Robert Audley found the driver asleep upon the...
1,1946,8,"""South America, Take It Away""",Bing Crosby and The Andrews Sisters,"South America, Take It Away",Bing Crosby,Up here in the land of the hot-dog stand\nThe ...


In [18]:
a = "A and B & C with D featuring E"
a.replace(" featuring ", "|").replace(" and ", "|").replace(" & ", "|").replace(" with ", "|")

'A|B|C|D|E'

In [37]:
df = pd.read_csv("data/lyrics0~9.csv")
df

,year,rank,title,artist,modified_title,modified_artist,lyric
0,1946,1,"""Prisoner of Love""",Perry Como,Prisoner of Love,Perry Como,Alone from night to night you'll find me|Too w...
1,1946,2,"""To Each His Own""",Eddy Howard,To Each His Own,Eddy Howard,A rose must remain with the sun and the rain|O...
2,1946,3,"""The Gypsy""",The Ink Spots,The Gypsy,The Ink Spots,In a quaint caravan|There's a lady they call t...
3,1946,4,"""Five Minutes More""",Frank Sinatra,Five Minutes More,Frank Sinatra,"Dear, this evening seemed to go so awfully fas..."
4,1946,5,"""Rumors Are Flying""",Frankie Carle,Rumors Are Flying,Frankie Carle,Rumors are flying|That you've got me sighing|T...
5,1946,6,"""Oh! What It Seemed to Be""",Frankie Carle,Oh! What It Seemed to Be,Frankie Carle,unreliable
6,1946,7,"""Personality""",Johnny Mercer and The Pied Pipers,Personality,Johnny Mercer,When Madame Pompadour was on a ballroom floor|...
7,1946,8,"""South America, Take It Away""",Bing Crosby and The Andrews Sisters,"South America, Take It Away",Bing Crosby,Up here in the land of the hot-dog stand|The a...
8,1946,9,"""The Gypsy""",Dinah Shore,The Gypsy,Dinah Shore,In a quaint caravan|There's a lady they call T...
9,1946,10,"""Oh! What It Seemed to Be""",Frank Sinatra,Oh! What It Seemed to Be,Frank Sinatra,It was just a neighborhood dance|That's all th...
